### Objective 1
We want something which will, given the md trajectory, calculate the dihedral angles (from this we can get their distributions)
given the relevant SMILES string.

### Objective 2
We also want something to plot the csv file data, so that we can do a sanity check to ensure the macroscopic properties (temp, pressure)
are reasonable and are not discontinuous.

In [20]:
import mdtraj as md
import numpy as np
import os

In [ ]:
smiles_dict = {
    'Lys-Tyr': 'CNCc1c(O)ccc(c1)C',
    'Lys-Arg': 'N1(C)CN=C(NC1)NC',
    'Sulfur-Mediated-Amide': 'SC[C@@H](NC(=O)C)C(=O)C',
    'Carboxyl-Carboxyl': 'C(=O)(C)NCc1ccc(cc1)CNC(=O)C',
    'Disulfide': ' N[C@H](C(=O)O)CSSC[C@@H](N)C(=O)O',
    'Cys-Arg': 'CSCC(=O)NCCCC',
    'Cys-Carboxyl': 'CSCC(=O)C',
}

pdb_dir = '/home/bfd21/rds/hpc-work/tbg/md_work/cyc_toy_pdbs'

pdb_dict = {}